# Adding Memory to a Chain (Part 1): Implementing the Setup

In [ ]:
# Run the line of code below to check the version of langchain in the current environment.
# Substitute "langchain" with any other package name to check their version.

In [ ]:
pip show langchain

In [9]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.runnables import chain

from langchain_openai import ChatOpenAI

from langchain.memory import ConversationSummaryMemory

from operator import itemgetter

In [2]:
TEMPLATE = '''
The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{message_log}

Human: 
{question}

AI:
'''

prompt_template = PromptTemplate.from_template(template=TEMPLATE)

In [3]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
    task="text-generation",
    max_new_tokens=100,
    do_sample=False,
    repetition_penalty=1.03,
)

chat = ChatHuggingFace(llm=llm, verbose=True)

In [4]:
chain = prompt_template | chat | StrOutputParser()

In [5]:
chain.invoke({'message_log':'''The AI provides an interesting fact about octopuses, stating that they have three hearts. 
Two hearts pump blood to the gills, while the third heart pumps it to the rest of the body.''', 
              'question':"Can you elaborate a bit more on this fact?"})

"I'd be happy to dive deeper into the fascinating anatomy of octopuses!\n\nYou see, octopuses have evolved to be highly efficient predators, and their three hearts play a crucial role in their unique physiology. The two branchial hearts, located near the base of the octopus's arms, pump blue-green blood (which is different from the red blood found in humans) to the octopus's gills, allowing them to extract oxygen from the water. These hearts work in a coordinated manner"

In [6]:
chat_memory = ConversationSummaryMemory(llm = ChatHuggingFace(llm=llm), memory_key = 'message_log')

C:\Users\Akaash\AppData\Local\Temp\ipykernel_24628\1394549114.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  chat_memory = ConversationSummaryMemory(llm = ChatHuggingFace(llm=llm), memory_key = 'message_log')


In [12]:
chat_memory.load_memory_variables({})

{'message_log': ''}

In [15]:
@chain
def memory_chain(question):
    
    chain1 = (
        RunnablePassthrough.assign(
            message_log = RunnableLambda(chat_memory.load_memory_variables) | 
            itemgetter('message_log')) 
        | prompt_template 
        | chat 
        | StrOutputParser()
    )
    
    response = chain1.invoke({'question':question})

    chat_memory.save_context(inputs = {'input':question}, 
                             outputs = {'output':response})

    return response

In [17]:
memory_chain.invoke("Tell me about a Ruskin Bond book related to Trains")

'What a great topic! Ruskin Bond is a beloved Indian author known for his children\'s books and short stories. He has written several books that feature trains as a significant part of the narrative.\n\nOne of his most famous books related to trains is "The Train from Turning" (1972). The story revolves around a young boy, Raghu, who boards the Turning Railway train to visit his grandfather in Dehradun. The train journey becomes a magical experience for Raghu, filled with encounters with'